In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import psrc_theme

%matplotlib inline
from IPython.display import display, HTML

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))
valid_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))

# create connection
conn = create_engine('sqlite:///../../../../inputs/db/'+input_config['db_name'])

# summary data location
csv_path = os.path.join(valid_config['model_dir'], 'outputs/validation')

# table format
pd.options.display.float_format = '{:0,.2f}'.format
format_percent = "{:,.2%}".format

# Link Expansion Analysis
Testing 2023 base year with Lynnwood extension and 2-Line connection to Seattle

### Line Level Totals


In [9]:
df_obs = pd.read_sql(
    "SELECT * FROM light_rail_station_boardings WHERE year IN (2023, 2025)",
    con=conn,
)
df_obs.rename(columns={"boardings": "observed_boardings"}, inplace=True)

df_obs
df_obs_pivot = pd.pivot_table(df_obs, values='observed_boardings', columns='year', index='station_name', aggfunc=np.sum)
df_obs_pivot = df_obs_pivot.fillna(0)
df_obs_pivot = df_obs_pivot.reset_index()
_df = df_obs.loc[df_obs['year']==2025,['station_name','emme_node']]
df_obs_pivot = df_obs_pivot.merge(_df, on=['station_name'])

df = pd.read_csv(os.path.join(valid_config['model_dir'], 'outputs/transit/boardings_by_stop.csv'))
df = df[df["i_node"].isin(df_obs["emme_node"])]
df = df.merge(df_obs_pivot, left_on="i_node", right_on="emme_node")
df.rename(columns={"total_boardings": "Model Boardings"}, inplace=True)
df.index = df["station_name"]
df_total = df.copy()[["Model Boardings",2023,2025, ]]
df_total.loc["Total", ["Model Boardings", 2023,2025]] = (
    df[["Model Boardings", 2023,2025]].sum().values
)
df_total
df_total.rename(columns={2023: 'Observed Spring 2023 (Pre-Lynnwood)', 2025: 'Observed Fall 2024 (Post-Lynnwood)' }, 
               inplace=True)
line_1_stations = ["Lynnwood Station","Mountlake Terrace Station","185th St Station","145th St Station",
                "Northgate Station",'Roosevelt Station','University District Station','Husky Stadium',
              'Capitol Hill','Westlake','Symphony',
              "Pioneer Square","Int'l District",
                 'Stadium','SODO','Beacon Hill','Mount Baker','Columbia City','Othello','Rainier Beach',
                "Tukwila Int'l Blvd",'SeaTac/Airport','Angle Lake']
df_1_line = df_total.loc[line_1_stations]

line_2_stations = ['South Bellevue Station','East Main Station','Bellevue Station','Wilburton Station',
             'Spring District/120th Station','Bel-Red/130th Station','Overlake Village Station',
             'Redmond Technology Station']
df_2_line = df_total.loc[line_2_stations]

In [10]:
df_total.loc[df_total.index.isin(line_1_stations),'Line'] = '1-Line'
df_total.loc[df_total.index.isin(line_2_stations),'Line'] = '2-Line'
df_total.groupby('Line').sum()

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
Line,,,
1-Line,"94,117.53","79,522.00","92,766.00"
2-Line,"7,269.17",0.00,"5,422.00"


### 1-Line Lynnwood to Angle Lake

In [11]:
df_1_line

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
station_name,,,
Lynnwood Station,"6,515.06",0.00,"5,146.00"
Mountlake Terrace Station,"3,083.54",0.00,"1,393.00"
185th St Station,"4,008.04",0.00,957.00
145th St Station,"1,945.94",0.00,"1,020.00"
Northgate Station,"5,963.15","10,384.00","4,150.00"
Roosevelt Station,"8,297.45","4,349.00","3,693.00"
University District Station,"7,305.50","7,563.00","6,004.00"
Husky Stadium,"4,140.85","6,134.00","5,122.00"
Capitol Hill,"7,479.32","7,667.00","8,207.00"


In [12]:
fig = px.line(df_1_line)
fig.update_layout(height=500, width=900, font=dict(size=11),
                  yaxis_tickformat = '.00f')
fig.show()

### 2-Line

In [13]:
df_2_line

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
station_name,,,
South Bellevue Station,538.92,0.00,"1,031.00"
East Main Station,697.39,0.00,234.00
Bellevue Station,"1,557.92",0.00,"1,212.00"
Wilburton Station,473.15,0.00,293.00
Spring District/120th Station,309.34,0.00,424.00
Bel-Red/130th Station,"1,421.96",0.00,311.00
Overlake Village Station,"1,305.30",0.00,418.00
Redmond Technology Station,965.21,0.00,"1,499.00"


In [14]:
fig = px.line(df_2_line)
fig.update_layout(height=500, width=900, font=dict(size=11),
                  yaxis_tickformat = '.00f')
fig.show()